In [1]:
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender/notebooks'

In [2]:
import os
os.chdir('../')
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender'

In [41]:
from src.job_recommender.chains.llm_chain import JobRecommenderChain
from src.job_recommender.schema.output_schema import *
from src.job_recommender.models.llm_model import LLMJobAssistant

In [30]:
resume_summary_chain = JobRecommenderChain(
    schema=ResumeSummary).chain

In [5]:
keywords_chain = JobRecommenderChain(
    schema=Keywords).chain

In [42]:
find_skill_gap_chain = JobRecommenderChain(
    schema=FindSkillGap).chain

In [49]:
find_skill_gap_chain.invoke({
    "file_path": "prompts/finding_skill_gap_prompt.txt",
    "resume_text": sample_resume
})

[2025-08-05 22:45:36,702|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 400 Bad Request"]


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n  "skills": ["Kubernetes", "Cloud AI Platform", "Cloud Storage", "Cloud Functions", "Explainable AI"],\n  "recommendations": ["Take online courses on Kubernetes and Cloud AI Platform to gain hands-on experience. Use cloud storage services like Google Cloud Storage or AWS S3 to store and manage data. Learn about Cloud Functions and how to deploy machine learning models using serverless computing. Read articles and research papers on Explainable AI to understand how to apply it to your machine learning projects."]\n}\n</tool-use>'}}

In [7]:
project_idea_chain = JobRecommenderChain(
    schema=ProjectIdeas).chain

In [8]:
improvement_areas_chain = JobRecommenderChain(
    schema=ImprovementAreas).chain

In [9]:
from langchain.schema.runnable import RunnableParallel
from langchain_core.runnables import RunnableLambda

In [10]:
def create_path(folder_path,file_name):
    return os.path.join(folder_path,file_name)

In [12]:
def safe_dict_access(x, key):
    # print(x,key)
    if not isinstance(x, dict):
        raise TypeError(f"Input to chain must be a dict, got {type(x)}")
    return x[key]


summary_chain = RunnableLambda(
    lambda x: resume_summary_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "summary_prompt.txt"),
        "resume_text": safe_dict_access(x, "resume")
    })
)

keyword_chain = RunnableLambda(
    lambda x: keywords_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "keyword_prompt.txt"),
        "summary": safe_dict_access(x, "summary")
    })
)


In [13]:
from langchain_core.runnables import RunnableSequence

In [14]:
def check_summary(x):
    # pr(type(x))
    print(x)

In [14]:
job_recommendation_chain = RunnableSequence(
    lambda x: {
        "folder_path": x["folder_path"],  # store it for later
        "summary": summary_chain.invoke(x)
    },
    lambda x: keyword_chain.invoke({
        "folder_path": x['folder_path'],
        "summary": x["summary"]
    })
)

In [15]:
sample_resume = LLMJobAssistant._load_file("sample_resume.txt")

In [16]:
job_recommendation_chain.invoke({
    "folder_path": "prompts/",
    "resume": sample_resume,
})

[2025-08-05 22:18:01,724|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]
[2025-08-05 22:18:02,661|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]


Keywords(keywords=['Machine Learning Engineer', 'Data Science', 'Python', 'TensorFlow', 'PyTorch', 'Scikit-learn'])

In [24]:
skill_gap_chain = RunnableLambda(
    lambda x: find_skill_gap_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "finding_skill_gap_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

projects_idea_chain = RunnableLambda(
    lambda x: project_idea_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "project_ideas_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

improvements_areas_chain = RunnableLambda(
    lambda x: improvement_areas_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "wycd_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

In [25]:
suggestions_chain = RunnableParallel({
        "find_skill_gap": skill_gap_chain, 
        "project_ideas": projects_idea_chain,
        "improvement_areas": improvements_areas_chain
    })     

In [26]:
suggestions_chain.invoke({
    "folder_path": "prompts/",
    "resume": sample_resume
})

[2025-08-05 22:21:26,642|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 400 Bad Request"]
[2025-08-05 22:21:27,101|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]
[2025-08-05 22:21:27,104|(INFO)| File: _client | Message: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': "Based on the provided resume, here are the top 5 missing skills that are currently in demand in the job market:\n\n1. **Cloud Computing**: The resume mentions experience with cloud platforms (AWS, Azure), but it would be beneficial to have more comprehensive knowledge of cloud computing, including migration strategies, cost optimization, and security best practices.\n\t* Recommendation: Take online courses on cloud computing platforms (e.g., AWS, Azure, Google Cloud) and obtain certifications (e.g., AWS Certified Cloud Practitioner).\n2. **DevOps**: The resume mentions experience with Docker and Git, but a deeper understanding of DevOps practices, including continuous integration and delivery, would be valuable.\n\t* Recommendation: Take online courses

In [23]:
job_recommendation_chain.get_graph().print_ascii()  # Display the graph of the final chain

    +-------------+      
    | LambdaInput |      
    +-------------+      
            *            
            *            
            *            
    +--------------+     
    | _load_prompt |     
    +--------------+     
            *            
            *            
            *            
+--------------------+   
| ChatPromptTemplate |   
+--------------------+   
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+---------------------+  
| PydanticToolsParser |  
+---------------------+  
            *            
            *            
            *            
    +--------------+     
    | _load_prompt |     
    +--------------+     
            *            
            *            
            *            
+--------------------+   
| ChatPromptTemplate |   
+-----------